## Phase 1: Data Sanitation and Preprocessing
This phase involves loading the Online Retail II dataset into a Pandas DataFrame, cleaning it by removing duplicates, missing Customer IDs, cancelled orders, zero-price rows, and non-product StockCodes, creating new features (TotalPrice, temporal columns), and converting data types as per the project specification (pages 7-8).

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from datetime import datetime, timedelta


# Set plot style for visualizations
sns.set(style="whitegrid")
%matplotlib inline

### Step 1: Load Dataset
The `online_retail.csv` file is loaded into a Pandas DataFrame. Initial inspection includes checking the shape, data types, and missing values to understand the dataset's structure.

In [12]:
# Load dataset
df = pd.read_csv('D:/Python/Python Final CW/online_retail.csv', encoding='latin1')  # Use latin1 if encoding issues occur
print("Initial Dataset Shape:", df.shape)
print("\nFirst 5 Rows:\n", df.head())
print("\nData Types and Non-Null Counts:\n")
df.info()

print()
print()
print("\nMissing Values:\n", df.isnull().sum())

Initial Dataset Shape: (1067371, 8)

First 5 Rows:
   Invoice StockCode                          Description  Quantity  \
0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
1  489434    79323P                   PINK CHERRY LIGHTS        12   
2  489434    79323W                  WHITE CHERRY LIGHTS        12   
3  489434     22041         RECORD FRAME 7" SINGLE SIZE         48   
4  489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24   

           InvoiceDate  Price  Customer ID         Country  
0  2009-12-01 07:45:00   6.95      13085.0  United Kingdom  
1  2009-12-01 07:45:00   6.75      13085.0  United Kingdom  
2  2009-12-01 07:45:00   6.75      13085.0  United Kingdom  
3  2009-12-01 07:45:00   2.10      13085.0  United Kingdom  
4  2009-12-01 07:45:00   1.25      13085.0  United Kingdom  

Data Types and Non-Null Counts:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column    

### Step 2: Remove Duplicates
Duplicate rows are identified and removed to ensure each transaction is unique, as required by the specification (page 7). This prevents over-counting in analysis.

In [13]:
# Check and remove duplicates
print("Duplicate Rows:", df.duplicated().sum())
df = df.drop_duplicates()
print("Shape after removing duplicates:", df.shape)

Duplicate Rows: 34335
Shape after removing duplicates: (1033036, 8)


### Step 3: Handle Missing Customer IDs
Rows with missing Customer IDs are removed, as they are critical for RFM analysis (page 7). This decision may exclude ~25% of data, potentially affecting analysis, which will be discussed in the report.

In [14]:
# Remove missing Customer IDs
print("Missing Customer IDs:", df['Customer ID'].isnull().sum())
df = df.dropna(subset=['Customer ID'])
print("Shape after removing missing Customer IDs:", df.shape)

Missing Customer IDs: 235151
Shape after removing missing Customer IDs: (797885, 8)


### Step 4: Clean Transactional Data
This step removes:
- Cancelled orders (Invoice starting with 'C') as they do not represent valid sales (page 5).
- Zero-price rows, which may indicate errors or promotional items (page 7).
- Non-product StockCodes (e.g., postage, bank fees) to focus on actual product sales (page 7).

In [15]:
# Remove cancelled orders
df = df[~df['Invoice'].str.startswith('C', na=False)]
print("Shape after removing cancellations:", df.shape)

# Remove zero-price rows
print("Zero Price Rows:", len(df[df['Price'] == 0]))
df = df[df['Price'] > 0]
print("Shape after removing zero-price rows:", df.shape)

# Remove non-product StockCodes
non_product_codes = ['POST', 'DOT', 'M', 'BANK CHARGES', 'C2', 'CRUK']
df = df[~df['StockCode'].isin(non_product_codes)]
print("Shape after removing non-product StockCodes:", df.shape)

Shape after removing cancellations: (779495, 8)
Zero Price Rows: 70
Shape after removing zero-price rows: (779425, 8)
Shape after removing non-product StockCodes: (776646, 8)


### Step 5: Feature Engineering
A `TotalPrice` column is created by multiplying Quantity and Price to measure revenue. Temporal features (Year, Month, DayOfWeek, HourOfDay) are extracted from InvoiceDate for time-based analysis (page 8).

In [16]:
# Create TotalPrice column
df['TotalPrice'] = df['Quantity'] * df['Price']

# Extract temporal features
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['Year'] = df['InvoiceDate'].dt.year
df['Month'] = df['InvoiceDate'].dt.month
df['DayOfWeek'] = df['InvoiceDate'].dt.dayofweek  # 0=Monday, 6=Sunday
df['HourOfDay'] = df['InvoiceDate'].dt.hour

print("First 5 Rows with New Columns:\n", df.head())

First 5 Rows with New Columns:
   Invoice StockCode                          Description  Quantity  \
0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
1  489434    79323P                   PINK CHERRY LIGHTS        12   
2  489434    79323W                  WHITE CHERRY LIGHTS        12   
3  489434     22041         RECORD FRAME 7" SINGLE SIZE         48   
4  489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24   

          InvoiceDate  Price  Customer ID         Country  TotalPrice  Year  \
0 2009-12-01 07:45:00   6.95      13085.0  United Kingdom        83.4  2009   
1 2009-12-01 07:45:00   6.75      13085.0  United Kingdom        81.0  2009   
2 2009-12-01 07:45:00   6.75      13085.0  United Kingdom        81.0  2009   
3 2009-12-01 07:45:00   2.10      13085.0  United Kingdom       100.8  2009   
4 2009-12-01 07:45:00   1.25      13085.0  United Kingdom        30.0  2009   

   Month  DayOfWeek  HourOfDay  
0     12          1          7  
1     

### Step 6: Data Type Conversion
Converting:
- InvoiceDate to datetime64[ns] for temporal analysis.
- Customer ID to integer after removing missing values.
- StockCode to string as it’s alphanumeric and not used for calculations (page 8).

In [17]:
# Convert data types
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])  # Ensure datetime
df['Customer ID'] = df['Customer ID'].astype(int)
df['StockCode'] = df['StockCode'].astype(str)

print("\nData Types After Conversion:\n")
df.info()


Data Types After Conversion:

<class 'pandas.core.frame.DataFrame'>
Index: 776646 entries, 0 to 1067369
Data columns (total 13 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      776646 non-null  object        
 1   StockCode    776646 non-null  object        
 2   Description  776646 non-null  object        
 3   Quantity     776646 non-null  int64         
 4   InvoiceDate  776646 non-null  datetime64[ns]
 5   Price        776646 non-null  float64       
 6   Customer ID  776646 non-null  int64         
 7   Country      776646 non-null  object        
 8   TotalPrice   776646 non-null  float64       
 9   Year         776646 non-null  int32         
 10  Month        776646 non-null  int32         
 11  DayOfWeek    776646 non-null  int32         
 12  HourOfDay    776646 non-null  int32         
dtypes: datetime64[ns](1), float64(2), int32(4), int64(2), object(4)
memory usage: 71.1+ MB


### Step 7: Final Dataset Check
Verifying the cleaned dataset’s shape, structure, and absence of critical missing values to ensure it’s ready for analysis in subsequent phases.

In [18]:
print("Final Cleaned Dataset Shape:", df.shape)
print("\nSample Data:\n", df.head())
print("\nMissing Values After Cleaning:\n", df.isnull().sum())

Final Cleaned Dataset Shape: (776646, 13)

Sample Data:
   Invoice StockCode                          Description  Quantity  \
0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
1  489434    79323P                   PINK CHERRY LIGHTS        12   
2  489434    79323W                  WHITE CHERRY LIGHTS        12   
3  489434     22041         RECORD FRAME 7" SINGLE SIZE         48   
4  489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24   

          InvoiceDate  Price  Customer ID         Country  TotalPrice  Year  \
0 2009-12-01 07:45:00   6.95        13085  United Kingdom        83.4  2009   
1 2009-12-01 07:45:00   6.75        13085  United Kingdom        81.0  2009   
2 2009-12-01 07:45:00   6.75        13085  United Kingdom        81.0  2009   
3 2009-12-01 07:45:00   2.10        13085  United Kingdom       100.8  2009   
4 2009-12-01 07:45:00   1.25        13085  United Kingdom        30.0  2009   

   Month  DayOfWeek  HourOfDay  
0     12      